In [330]:
import numpy as np
import pandas as pd

In [331]:
first_df1 = pd.read_csv("modeling data/1_players_info_added_Clean_data.csv")
df1_nulls = first_df1.isnull().sum()
print(df1_nulls)
print(len(first_df1))

Unnamed: 0           0
Location             0
Date                 0
Round                0
Winner               0
Loser                0
WRank               60
LRank              206
W_hand               0
W_ht               957
W_nationality      858
W_age              859
L_hand               0
L_ht              1102
L_nationality      791
L_age              796
L_wt             11818
W_wt             11801
dtype: int64
19922


In [332]:
df_drop_nulls = first_df1.copy()
df_drop_nulls = df_drop_nulls.dropna()
len(df_drop_nulls)

7399

In [333]:
df1_dropped_wt = first_df1.copy()
df1_dropped_wt.drop(columns=["Unnamed: 0",'L_wt', 'W_wt'], inplace=True)
df1_no_nulls = df1_dropped_wt.dropna()
len(df1_no_nulls)

17917

In [258]:
df_winner = df1_no_nulls.copy()
df_loser = df1_no_nulls.copy()

df_winner['win'] = 1
df_loser['win'] = 0

# Renaming columns to 'player' and 'opponent' for winners
df_winner.rename(columns={
    'Winner': 'player',
    'Loser': 'opponent',
    'WRank': 'player_rank',
    'LRank': 'opponent_rank',
    'W_hand': 'player_hand',
    'L_hand': 'opponent_hand',
    'W_ht': 'player_ht', 
    'W_nationality':'player_nationality', 
    'W_age':'player_age', 
    'L_ht':'opponent_ht',
    'L_nationality':'opponent_nationality', 
    'L_age':'opponent_age'
}, inplace=True)

# Renaming columns to 'player' and 'opponent' for losers
df_loser.rename(columns={
    'Winner': 'opponent',
    'Loser': 'player',
    'WRank': 'opponent_rank',
    'LRank': 'player_rank',
    'W_hand': 'opponent_hand',
    'L_hand': 'player_hand',
    'W_ht':'opponent_ht', 
    'W_nationality':'opponent_nationality', 
    'W_age':'opponent_age', 
    'L_ht':'player_ht',
    'L_nationality':'player_nationality',
    'L_age':'player_age'
}, inplace=True)
df_transformed = pd.concat([df_winner, df_loser], ignore_index=True)
df_transformed = df_transformed.sample(frac=1).reset_index(drop=True)

In [259]:
player_peak_ranking = df_transformed.groupby('player')['player_rank'].min().reset_index()
opponent_peak_ranking = df_transformed.groupby('opponent')['opponent_rank'].min().reset_index()
player_peak_ranking

player_updated = pd.merge(df_transformed, player_peak_ranking[['player', 'player_rank']], on='player', how='left')
opponent_updated = pd.merge(player_updated, opponent_peak_ranking[['opponent', 'opponent_rank']], on='opponent', how='left')
df1 = opponent_updated.copy()
df1.drop(columns=['player_rank_x', 'opponent_rank_x'], inplace=True)
df1 = df1.rename(columns={  'player_rank_y': 'player_rank',
                            'opponent_rank_y': 'opponent_rank'})
df1

,Location,Date,Round,player,opponent,player_hand,player_ht,player_nationality,player_age,opponent_hand,opponent_ht,opponent_nationality,opponent_age,win,player_rank,opponent_rank
0,Lyon,5/21/2018,Round of 128,Zeballos H.,Lajovic D.,L,188.0,ARG,39.0,R,180.0,SRB,28.5,0,49.000000,23.000000
1,Melbourne,1/18/2017,Round of 64,Seppi A.,Kyrgios N.,R,190.0,ITA,34.8,R,193.0,AUS,23.6,1,35.000000,13.000000
2,Chennai,1/5/2017,Round of 64,Paire B.,Bhambri Y.,R,196.0,FRA,29.6,R,183.0,IND,32.0,1,19.000000,84.000000
3,Roland Garros,9/28/2020,Round of 32,Berrettini M.,Altmaier D.,R,196.0,ITA,24.4,R,191.0,GER,22.0,0,8.000000,122.000000
4,australian-open,1/1/2016,1st Round Qualifying,Molchanov D.,Mousley B.,R,191.0,UKR,37.0,R,188.0,AUS,28.0,1,483.283333,1008.685106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,Pune,2/3/2020,Quarterfinals,Marcora R.,Duckworth J.,R,194.0,ITA,30.4,R,183.0,AUS,28.0,0,174.000000,84.000000
35830,New York,8/27/2018,Round of 128,Norrie C.,Thompson J.,L,188.0,GBR,23.3,R,183.0,AUS,24.6,1,41.000000,43.000000
35831,Vienna,10/21/2019,Quarterfinals,Rublev A.,Berrettini M.,R,188.0,RUS,22.0,R,196.0,ITA,23.5,0,8.000000,8.000000
35832,Basel,10/21/2019,Semifinals,Federer R.,Tsitsipas S.,R,185.0,SUI,38.2,R,193.0,GRE,21.1,1,1.000000,5.000000


In [260]:
countryList = [
    ["AUS", "Australia", "Canberra", "Sydney", "Melbourne", "Brisbane", "Perth", "Adelaide"],
    ["IND", "India", "New Delhi", "Mumbai", "Kolkata", "Chennai", "Bangalore", "Hyderabad", "Pune"],
    ["QAT", "Qatar", "Doha"],
    ["NZL", "New Zealand", "Wellington", "Auckland", "Christchurch"],
    ["FRA", "France", "Paris", "Marseille", "Lyon", "Toulouse", "Nice", "Nantes", "Strasbourg", "Montpellier", "Bordeaux", "Lille","Roland Garros","Metz"],
    ["ECU", "Ecuador", "Quito", "Guayaquil"],
    ["BGR", "Bulgaria", "Sofia", "Plovdiv", "Varna"],
    ["ARG", "Argentina", "Buenos Aires", "Cordoba", "Rosario", "Mendoza"],
    ["USA", "US", "United States", "New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia", "San Antonio", "San Diego", "Dallas", "Salem", "Cincinnati","Newport",'Atlanta','Washington','Memphis',"Delray Beach","Miami"],
    ["BRA", "Brazil", "Brasília", "São Paulo", "Sao Paulo","Rio de Janeiro","Rio", "Salvador", "Fortaleza", "Belo Horizonte"],
    ["MEX", "Mexico", "Mexico City", "Guadalajara", "Monterrey", "Puebla", "Toluca", "Tijuana","Acapulco","Los Cabos"],
    ["ARE", "United Arab Emirates", "Abu Dhabi", "Dubai", "Sharjah"],
    ["MAR", "Morocco", "Rabat", "Casablanca", "Fez", "Marrakech"],
    ["MCO", "Monaco", "Monte Carlo"],
    ["ESP", "Spain", "Madrid", "Barcelona", "Valencia", "Seville", "Zaragoza"],
    ["HUN", "Hungary", "Budapest"],
    ["PRT", "Portugal", "Lisbon", "Porto", "Amadora", "Braga","Estoril"],
    ["TUR", "Turkey", "Ankara", "Istanbul", "Izmir", "Bursa", "Antalya"],
    ["DEU", "Germany", "Berlin", "Hamburg", "Munich", "Cologne", "Frankfurt", "Stuttgart", "Düsseldorf","Dusseldorf", "Dortmund", "Essen", "Leipzig","Halle"],
    ["ITA", "Italy", "Rome", "Milan", "Naples", "Turin", "Palermo", "Genoa", "Bologna", "Florence","nitto","Sardinia"],
    ["CHE", "Switzerland", "Zurich", "Geneva", "Basel", "Lausanne","Gstaad"],
    ["NLD", "Netherlands", "Amsterdam", "Rotterdam", "The Hague", "Utrecht","s Hertogenbosch"],
    ["GBR", "United Kingdom", "London", "Birmingham", "Glasgow", "Liverpool", "Leeds", "Sheffield", "Edinburgh", "Bristol", "Manchester","Eastbourne","wimbledon"],
    ["SWE", "Sweden", "Stockholm", "Gothenburg", "Malmö","Malmo", "Uppsala",'bastad'],
    ["HRV", "Croatia", "Zagreb", "Split", "Rijeka",'Umag'],
    ["CHN", "China", "Beijing", "Shanghai", "Chongqing", "Tianjin", "Guangzhou", "Shenzhen", "Chengdu","Zhuhai"],
    ["BEL", "Belgium", "Brussels", "Antwerp", "Ghent", "Charleroi", "Liège"],
    ["RUS", "Russia", "Moscow", "Saint Petersburg","st petersburg","Petersburg" "Novosibirsk", "Yekaterinburg", "Nizhny Novgorod"],
    ["CHL", "Chile", "Santiago"],
    ["CAN", "Canada", "Ottawa", "Toronto", "Montreal", "Calgary", "Edmonton", "Vancouver", "Winnipeg", "Quebec City", "Hamilton", "Kitchener"],
    ["KAZ", "Kazakhstan", "Nur Sultan", "Almaty"],
    ["AUT", "Austria", "Vienna","Kitzbuhel"],
    ["JPN", "Japan", "Tokyo", "Yokohama", "Osaka", "Nagoya", "Sapporo", "Fukuoka", "Kobe", "Kyoto"],
    ["Unknown", "Davis","Club",'Cup', 'Tour Finals',"NextGen"]
]

In [261]:
import re

newCountryName = df1.copy()
newCountryName["Location"] = newCountryName["Location"].str.replace('_'," ")
newCountryName["Location"] = newCountryName["Location"].str.replace('-'," ")

for country in countryList:
    for i in range(len(country)):
        newCountryName["Location"] = newCountryName["Location"].str.replace('.*'+country[i]+'.*', country[0], flags=re.IGNORECASE, regex=True)
newCountryName["Location"].unique()

array(['FRA', 'USA', 'IND', 'DEU', 'NLD', 'ARE', 'CHE', 'TUR', 'ITA',
       'ECU', 'RUS', 'GBR', 'ARG', 'MCO', 'ESP', 'MAR', 'MEX', 'JPN',
       'BEL', 'Unknown', 'CHN', 'BGR', 'BRA', 'CAN', 'AUT', 'KAZ', 'NZL',
       'CHL', 'QAT', 'HRV', 'SWE', 'HUN', 'PRT'], dtype=object)

In [263]:
df1_home_country = newCountryName.copy()
df1_home_country['is_player_home_country'] = np.equal(df1_home_country['Location'], df1_home_country['player_nationality'])
df1_home_country['is_opponent_home_country'] = np.equal(df1_home_country['Location'], df1_home_country['opponent_nationality'])
df1_home_country.drop(columns=["Location",'player_nationality', 'opponent_nationality'], inplace=True)
df1_home_country

,Date,Round,player,opponent,player_hand,player_ht,player_age,opponent_hand,opponent_ht,opponent_age,win,player_rank,opponent_rank,is_player_home_country,is_opponent_home_country
0,5/21/2018,Round of 128,Zeballos H.,Lajovic D.,L,188.0,39.0,R,180.0,28.5,0,49.000000,23.000000,False,False
1,1/18/2017,Round of 64,Seppi A.,Kyrgios N.,R,190.0,34.8,R,193.0,23.6,1,35.000000,13.000000,False,False
2,1/5/2017,Round of 64,Paire B.,Bhambri Y.,R,196.0,29.6,R,183.0,32.0,1,19.000000,84.000000,False,True
3,9/28/2020,Round of 32,Berrettini M.,Altmaier D.,R,196.0,24.4,R,191.0,22.0,0,8.000000,122.000000,False,False
4,1/1/2016,1st Round Qualifying,Molchanov D.,Mousley B.,R,191.0,37.0,R,188.0,28.0,1,483.283333,1008.685106,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,2/3/2020,Quarterfinals,Marcora R.,Duckworth J.,R,194.0,30.4,R,183.0,28.0,0,174.000000,84.000000,False,False
35830,8/27/2018,Round of 128,Norrie C.,Thompson J.,L,188.0,23.3,R,183.0,24.6,1,41.000000,43.000000,False,False
35831,10/21/2019,Quarterfinals,Rublev A.,Berrettini M.,R,188.0,22.0,R,196.0,23.5,0,8.000000,8.000000,False,False
35832,10/21/2019,Semifinals,Federer R.,Tsitsipas S.,R,185.0,38.2,R,193.0,21.1,1,1.000000,5.000000,False,False


In [264]:
def split_date(df):
    df1 = df.copy()
    df1['Date'] = pd.to_datetime(df1['Date'])
    df1['year'] = df1['Date'].dt.year
    df1['month'] = df1['Date'].dt.month
    df1['day'] = df1['Date'].dt.day
    df1['dayofweek'] = df1['Date'].dt.dayofweek
    df1.drop(columns=["Date"], inplace=True)
    return df1

In [265]:
split_date(df1_home_country).to_csv('modeling data/df1_with_home.csv', index=False)


In [294]:
df_home = pd.read_csv("modeling data/df1_with_home.csv")
df_home

,Round,player,opponent,player_hand,player_ht,player_age,opponent_hand,opponent_ht,opponent_age,win,player_rank,opponent_rank,is_player_home_country,is_opponent_home_country,year,month,day,dayofweek
0,Round of 128,Zeballos H.,Lajovic D.,L,188.0,39.0,R,180.0,28.5,0,49.000000,23.000000,False,False,2018,5,21,0
1,Round of 64,Seppi A.,Kyrgios N.,R,190.0,34.8,R,193.0,23.6,1,35.000000,13.000000,False,False,2017,1,18,2
2,Round of 64,Paire B.,Bhambri Y.,R,196.0,29.6,R,183.0,32.0,1,19.000000,84.000000,False,True,2017,1,5,3
3,Round of 32,Berrettini M.,Altmaier D.,R,196.0,24.4,R,191.0,22.0,0,8.000000,122.000000,False,False,2020,9,28,0
4,1st Round Qualifying,Molchanov D.,Mousley B.,R,191.0,37.0,R,188.0,28.0,1,483.283333,1008.685106,False,False,2016,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,Quarterfinals,Marcora R.,Duckworth J.,R,194.0,30.4,R,183.0,28.0,0,174.000000,84.000000,False,False,2020,2,3,0
35830,Round of 128,Norrie C.,Thompson J.,L,188.0,23.3,R,183.0,24.6,1,41.000000,43.000000,False,False,2018,8,27,0
35831,Quarterfinals,Rublev A.,Berrettini M.,R,188.0,22.0,R,196.0,23.5,0,8.000000,8.000000,False,False,2019,10,21,0
35832,Semifinals,Federer R.,Tsitsipas S.,R,185.0,38.2,R,193.0,21.1,1,1.000000,5.000000,False,False,2019,10,21,0


In [322]:
df_outlier1 = df_home.copy()
def detect_outliers(df, column, threshold=4):
    """
    Detect outliers in a dataframe column using the IQR method.

    Returns:
        list: A list containing the indices of the outliers.
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)].index.tolist()
    return outliers


numeric_columns = ['player_ht', 'player_age', 'opponent_ht', 'opponent_age', 'player_rank', 'opponent_rank']
outliers = {}
outliers_count={}
for column in numeric_columns:
    outliers[column] = detect_outliers(df_outlier1, column)
    outliers_count[column] = len(outliers[column])


# Print out indices of outliers for each column
for column, indices in outliers.items():
    print(f"Outliers in column '{column}': {indices}, Count: {outliers_count[column]}")


Outliers in column 'player_ht': [], Count: 0
Outliers in column 'player_age': [], Count: 0
Outliers in column 'opponent_ht': [], Count: 0
Outliers in column 'opponent_age': [], Count: 0
Outliers in column 'player_rank': [4, 44, 50, 55, 108, 157, 174, 180, 193, 206, 229, 236, 257, 265, 302, 326, 330, 409, 441, 443, 451, 497, 524, 532, 552, 584, 606, 658, 660, 678, 695, 707, 761, 776, 808, 866, 898, 927, 960, 971, 1054, 1084, 1104, 1116, 1133, 1195, 1223, 1241, 1246, 1247, 1253, 1257, 1265, 1270, 1278, 1317, 1352, 1373, 1456, 1479, 1506, 1533, 1574, 1610, 1627, 1650, 1653, 1721, 1722, 1747, 1791, 1831, 1837, 1867, 1875, 1918, 1951, 2126, 2141, 2144, 2150, 2177, 2240, 2245, 2252, 2259, 2372, 2396, 2400, 2414, 2416, 2484, 2510, 2525, 2547, 2572, 2573, 2592, 2684, 2686, 2706, 2716, 2749, 2766, 2770, 2774, 2777, 2792, 2796, 2801, 2828, 2830, 2874, 2903, 2934, 2949, 2994, 2995, 3020, 3042, 3187, 3189, 3219, 3246, 3270, 3349, 3358, 3377, 3446, 3474, 3478, 3495, 3521, 3567, 3633, 3634, 3651, 36

In [296]:
from scipy import stats

df_outlier2 = df_home.copy()

import numpy as np

z_scores = np.abs(stats.zscore(df_outlier2[numeric_columns]))
filtered_entries = (z_scores < 3).all(axis=1)
df_outlier2 = df_outlier2[filtered_entries]
df_outlier2

,Round,player,opponent,player_hand,player_ht,player_age,opponent_hand,opponent_ht,opponent_age,win,player_rank,opponent_rank,is_player_home_country,is_opponent_home_country,year,month,day,dayofweek
0,Round of 128,Zeballos H.,Lajovic D.,L,188.0,39.0,R,180.0,28.5,0,49.0,23.0,False,False,2018,5,21,0
1,Round of 64,Seppi A.,Kyrgios N.,R,190.0,34.8,R,193.0,23.6,1,35.0,13.0,False,False,2017,1,18,2
2,Round of 64,Paire B.,Bhambri Y.,R,196.0,29.6,R,183.0,32.0,1,19.0,84.0,False,True,2017,1,5,3
3,Round of 32,Berrettini M.,Altmaier D.,R,196.0,24.4,R,191.0,22.0,0,8.0,122.0,False,False,2020,9,28,0
5,Round of 64,Lajovic D.,Troicki V.,R,183.0,34.0,R,193.0,38.0,0,23.0,29.0,False,False,2016,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,Quarterfinals,Marcora R.,Duckworth J.,R,194.0,30.4,R,183.0,28.0,0,174.0,84.0,False,False,2020,2,3,0
35830,Round of 128,Norrie C.,Thompson J.,L,188.0,23.3,R,183.0,24.6,1,41.0,43.0,False,False,2018,8,27,0
35831,Quarterfinals,Rublev A.,Berrettini M.,R,188.0,22.0,R,196.0,23.5,0,8.0,8.0,False,False,2019,10,21,0
35832,Semifinals,Federer R.,Tsitsipas S.,R,185.0,38.2,R,193.0,21.1,1,1.0,5.0,False,False,2019,10,21,0


In [297]:
#encoding the date using sin and cos to preserve the cyclical nature of
def encode_date(df):
    df_date_encoded = df_home.copy()
    df_date_encoded['month_sin'] = np.sin(2 * np.pi * df_date_encoded['month']/12)
    df_date_encoded['month_cos'] = np.cos(2 * np.pi * df_date_encoded['month']/12)
    df_date_encoded['dayofweek_sin'] = np.sin(2 * np.pi * df_date_encoded['dayofweek']/7)
    df_date_encoded['dayofweek_cos'] = np.cos(2 * np.pi * df_date_encoded['dayofweek']/7)
    df_date_encoded['day_sin'] = np.sin(2 * np.pi * df_date_encoded['day']/30.5)
    df_date_encoded['day_cos'] = np.cos(2 * np.pi * df_date_encoded['day']/30.5)
    df_date_encoded.drop(columns=["month","dayofweek", "day"], inplace=True)
    return df_date_encoded

In [298]:
#encoding rounds, no need to scale because the higher the round is, the more important it is
def encode_round(df):
    df_round_encoded = df.copy()
    round_mapping = {
        '1st Round Qualifying': 1,
        '2nd Round Qualifying': 2,
        '3rd Round Qualifying': 3,
        'Round Robin': 4,
        'Round of 128': 5,
        'Round of 64': 6,
        'Round of 32': 7,
        'Round of 16': 8,
        'Quarterfinals': 9,
        'Semifinals': 10,
        'The Final': 11
    }
    df_round_encoded['Round'] = df_round_encoded['Round'].map(round_mapping)
    return df_round_encoded

In [299]:
#binary encoding for columns in columnNames = ['is_player_home_country','is_opponent_home_country',"is_taller", "is_older", "rank_is_higher"]
def binary_column(df):
    df_binary = df.copy()
    df_binary['is_older'] = df_binary['player_age'] > df_binary['opponent_age']
    df_binary['is_taller'] = df_binary['player_ht'] > df_binary['opponent_ht']
    df_binary['rank_is_higher'] = df_binary['player_rank'] > df_binary['opponent_rank']

    df_binary.drop(columns=["player_age",'opponent_age', 'player_ht',"opponent_ht",'player_rank', 'opponent_rank'], inplace=True)
    return df_binary

In [300]:
#integer encoding for handedness
def int_encode_hand(df):
    df_hand_int_encoded = df.copy()
    hand_mapping = {
        'R': 1,
        'L': 2,
        'U': 0
    }
    df_hand_int_encoded['player_hand'] = df_hand_int_encoded['player_hand'].map(hand_mapping)
    df_hand_int_encoded['opponent_hand'] = df_hand_int_encoded['opponent_hand'].map(hand_mapping)
    return df_hand_int_encoded

In [301]:
#one-hot encoding for handedness
def hot_encode_hand(df):
    df_hand_1hot_encoded = df.copy()

    df_hand_1hot_encoded = pd.get_dummies(df_hand_1hot_encoded, columns=['player_hand'], prefix='player_hand')
    df_hand_1hot_encoded = pd.get_dummies(df_hand_1hot_encoded, columns=['opponent_hand'], prefix='opponent_hand')
    return df_hand_1hot_encoded

In [302]:
def encode_pipeline(df):
    df1 = df.copy()
    df2 = encode_date(df1)
    df3 = encode_round(df2)
    df_int_hand = int_encode_hand(df3)
    df_hot_hand = hot_encode_hand(df3)
    return df_int_hand,df_hot_hand

In [303]:
df_hand_int_encoded_home, df_hand_1hot_encoded_home = encode_pipeline(df_home)


df_hand_int_encoded_bool = binary_column(df_hand_int_encoded_home)
df_hand_1hot_encoded_bool = binary_column(df_hand_1hot_encoded_home)


In [304]:
#normalize function
numeric_columns_1 = ['player_ht', 'opponent_ht']
numeric_columns_2 = ['player_age','opponent_age']
numeric_columns_3 = ['player_rank', 'opponent_rank']
numeric_columns_4 = ["year"]

def normalize(og_df, numeric_columns):
    df = og_df.copy()
    for column in numeric_columns:
        # Z-score normalization
        df[f'{column}_zscore'] = (df[column] - df[column].mean()) / df[column].std()
        # Min-max scaling
        df[f'{column}_minmax'] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
        # Decimal scaling
        max_length = max(df[column].apply(lambda x: len(str(np.floor(x))) if x > 0 else 1))
        df[f'{column}_decimal'] = df[column] / (10**max_length)
    return df

In [316]:
#no normalization (NN) has home
df_hand_int_encoded_NN = df_hand_int_encoded_home.copy()
df_hand_1hot_encoded_NN = df_hand_1hot_encoded_home.copy()

#normalize all
df_hand_int_N_all = normalize(df_hand_int_encoded_NN,numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4)
df_hand_1hot_N_all = normalize(df_hand_1hot_encoded_NN,numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4)

df_hand_int_N_all.drop(columns=numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4, inplace=True)
df_hand_1hot_N_all.drop(columns=numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4, inplace=True)

df_hand_int_N_all_drop_year = df_hand_int_N_all.copy()
df_hand_1hot_N_all_drop_year = df_hand_1hot_N_all.copy()

df_hand_int_N_all_drop_year.drop(columns=["year_zscore","year_minmax","year_decimal"], inplace=True)
df_hand_1hot_N_all_drop_year.drop(columns=["year_zscore","year_minmax","year_decimal"], inplace=True)


#no normalization (NN) bool cols
df_hand_int_encoded_NN_bool = df_hand_int_encoded_bool.copy()
df_hand_1hot_encoded_NN_bool = df_hand_1hot_encoded_bool.copy()

#normalize year
df_hand_int_encoded_N_year_bool = normalize(df_hand_int_encoded_NN_bool,numeric_columns_4)
df_hand_1hot_encoded_N_year_bool = normalize(df_hand_1hot_encoded_NN_bool,numeric_columns_4)

df_hand_int_encoded_N_year_bool.drop(columns=numeric_columns_4, inplace=True)
df_hand_1hot_encoded_N_year_bool.drop(columns=numeric_columns_4, inplace=True)
df_hand_1hot_encoded_N_year_bool

#no normalization (NN) bool cols
df_hand_int_encoded_N_bool_drop_year = df_hand_int_encoded_NN_bool.copy()
df_hand_1hot_encoded_N_bool_drop_year = df_hand_1hot_encoded_NN_bool.copy()

#drop year
df_hand_int_encoded_N_bool_drop_year.drop(columns=numeric_columns_4, inplace=True)
df_hand_1hot_encoded_N_bool_drop_year.drop(columns=numeric_columns_4, inplace=True)

df_hand_int_encoded_bool

In [317]:
#df without outlier
df_hand_int_encoded_outlier, df_hand_1hot_encoded_outlier = encode_pipeline(df_outlier2)

df_hand_int_encoded_bool_outlier = binary_column(df_hand_int_encoded_outlier)
df_hand_1hot_encoded_bool_outlier = binary_column(df_hand_1hot_encoded_outlier)


#df without outlier
df_hand_int_encoded_bool_outlier_NN = df_hand_int_encoded_bool_outlier.copy()
df_hand_1hot_encoded_bool_outlier_NN = df_hand_1hot_encoded_bool_outlier.copy()

#normalize all
df_hand_int_N_all_outliers = normalize(df_hand_int_encoded_outlier,numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4)
df_hand_1hot_N_all_outliers = normalize(df_hand_1hot_encoded_outlier,numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4)

df_hand_int_N_all_outliers.drop(columns=numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4, inplace=True)
df_hand_1hot_N_all_outliers.drop(columns=numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4, inplace=True)

#normalize year
df_hand_int_encoded_N_year_bool_outliers = normalize(df_hand_int_encoded_bool_outlier_NN,numeric_columns_4)
df_hand_1hot_encoded_N_year_bool_outliers = normalize(df_hand_1hot_encoded_bool_outlier_NN,numeric_columns_4)

df_hand_int_encoded_N_year_bool_outliers.drop(columns=numeric_columns_4, inplace=True)
df_hand_1hot_encoded_N_year_bool_outliers.drop(columns=numeric_columns_4, inplace=True)



In [ ]:
df_no_nulls = df_drop_nulls.copy()

df_hand_int_encoded_nnull, df_hand_1hot_encoded_nnull = encode_pipeline(df_no_nulls)


df_hand_int_encoded_bool_nnull = binary_column(df_hand_int_encoded_nnull)
df_hand_1hot_encoded_bool_nnull = binary_column(df_hand_1hot_encoded_nnull)






#df without nulls
df_hand_int_encoded_bool_nnull_NN = df_hand_int_encoded_bool_nnull.copy()
df_hand_1hot_encoded_bool_nnull_NN = df_hand_1hot_encoded_bool_nnull.copy()

#normalize all
df_hand_int_N_all_nnull = normalize(df_hand_int_encoded_nnull,numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4)
df_hand_1hot_N_all_nnull = normalize(df_hand_1hot_encoded_nnull,numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4)

df_hand_int_N_all_nnull.drop(columns=numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4, inplace=True)
df_hand_1hot_N_all_nnull.drop(columns=numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4, inplace=True)

#normalize year
df_hand_int_encoded_N_year_bool_outliers = normalize(df_hand_int_encoded_bool_nnull_NN,numeric_columns_4)
df_hand_1hot_encoded_N_year_bool_outliers = normalize(df_hand_1hot_encoded_bool_nnull_NN,numeric_columns_4)

df_hand_int_encoded_N_year_bool_outliers.drop(columns=numeric_columns_4, inplace=True)
df_hand_1hot_encoded_N_year_bool_outliers.drop(columns=numeric_columns_4, inplace=True)

In [283]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

#split df into training and testing
def split_df(df):
    working_df = df.copy()
    X = working_df.drop(['win', 'player', 'opponent'], axis=1)  # Exclude non-numeric or target columns
    y = working_df['win']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    return X_train, X_test, y_train, y_test

#train Linear SVC model
def lin_svc_model(X_train, X_test, y_train, y_test):
    # Initialize the SVM classifier
    lsvm_model = LinearSVC(dual=True, max_iter=10000, random_state=1)
    lsvm_model.fit(X_train, y_train)
    y_pred_lsvm = lsvm_model.predict(X_test)
    print(classification_report(y_test, y_pred_lsvm))
    return y_test, y_pred_lsvm

#train SVC model
def svc_model(X_train, X_test, y_train, y_test):
    # Initialize the SVM classifier
    svm_model = SVC(kernel='linear', probability=True, random_state=1)
    svm_model.fit(X_train, y_train)
    y_pred_svm = svm_model.predict(X_test)
    print(classification_report(y_test, y_pred_svm))
    return y_test, y_pred_svm, svm_model, X_test

#train random forest model
def rfc_model(X_train, X_test, y_train, y_test):
    # Initialize the Random Forest classifier
    rf_model = RandomForestClassifier(n_estimators=100, random_state=1)
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_test)
    print(classification_report(y_test, y_pred_rf))
    return y_test, y_pred_rf, rf_model, X_test

#train logistic regression model
def logReg_model(X_train, X_test, y_train, y_test):
    # Initialize the Logistic Regression classifier
    lr_model = LogisticRegression(random_state=1)
    lr_model.fit(X_train, y_train)
    y_pred_lr = lr_model.predict(X_test)
    print(classification_report(y_test, y_pred_lr))
    return y_test, y_pred_lr, lr_model, X_test

In [276]:
# Calculate metrics
def calcMetric(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

def calcMetrics(y_test, y_pred, model, X_test):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])  # For binary classifications

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"ROC-AUC Score: {roc_auc}")

In [277]:
def lin_svc_pipeline(og_df):
    print("Linear Support Vector Classifier")
    df = og_df.copy()
    X_train, X_test, y_train, y_test = split_df(df)
    y_test, y_pred= lin_svc_model(X_train, X_test, y_train, y_test)
    calcMetric(y_test, y_pred)

def svc_pipeline(og_df):
    print("Support Vector Classifier")
    df = og_df.copy()
    X_train, X_test, y_train, y_test = split_df(df)
    y_test, y_pred, model, X_test = svc_model(X_train, X_test, y_train, y_test)
    calcMetrics(y_test, y_pred, model, X_test)

def rfc_pipeline(og_df):
    print("Random Forest Classifier")
    df = og_df.copy()
    X_train, X_test, y_train, y_test = split_df(df)
    y_test, y_pred, model, X_test = rfc_model(X_train, X_test, y_train, y_test)
    calcMetrics(y_test, y_pred, model, X_test)
    
def logReg_pipeline(og_df):
    print("Logistic regression Classifier")
    df = og_df.copy()
    X_train, X_test, y_train, y_test = split_df(df)
    y_test, y_pred, model, X_test = logReg_model(X_train, X_test, y_train, y_test)
    calcMetrics(y_test, y_pred, model, X_test)

In [318]:
# dataframes = [df_hand_int_encoded_NN,df_hand_1hot_encoded_NN,df_hand_int_N_all,df_hand_1hot_N_all]
# dataframe_names = ["hand_int_encoded_NN","hand_1hot_encoded_NN","hand_int_N_all","hand_1hot_N_all"]

dataframes = [df_hand_int_N_all,df_hand_1hot_N_all,df_hand_int_N_all_drop_year,df_hand_1hot_N_all_drop_year,
              df_hand_int_encoded_N_year_bool,df_hand_1hot_encoded_N_year_bool,df_hand_int_encoded_N_bool_drop_year,df_hand_1hot_encoded_N_bool_drop_year,
              df_hand_int_N_all_outliers,df_hand_1hot_N_all_outliers,df_hand_int_encoded_N_year_bool_outliers,df_hand_1hot_encoded_N_year_bool_outliers]
dataframe_names = ["df_hand_int_N_all","df_hand_1hot_N_all","df_hand_int_N_all_drop_year","df_hand_1hot_N_all_drop_year",
                   "df_hand_int_encoded_N_year_bool","df_hand_1hot_encoded_N_year_bool","df_hand_int_encoded_N_bool_drop_year","df_hand_1hot_encoded_N_bool_drop_year",
                   "df_hand_int_N_all_outliers","df_hand_1hot_N_all_outliers","df_hand_int_encoded_N_year_bool_outliers","df_hand_1hot_encoded_N_year_bool_outliers"]

In [319]:
df_hand_int_N_all

,Round,player,opponent,player_hand,opponent_hand,win,is_player_home_country,is_opponent_home_country,month_sin,month_cos,...,opponent_age_decimal,player_rank_zscore,player_rank_minmax,player_rank_decimal,opponent_rank_zscore,opponent_rank_minmax,opponent_rank_decimal,year_zscore,year_minmax,year_decimal
0,5,Zeballos H.,Lajovic D.,2,1,0,False,False,0.500000,-8.660254e-01,...,0.00285,-0.126635,0.023963,0.000049,-0.359042,0.010983,0.000023,0.247035,0.50,0.002018
1,6,Seppi A.,Kyrgios N.,1,1,1,False,False,0.500000,8.660254e-01,...,0.00236,-0.251778,0.016974,0.000035,-0.448430,0.005991,0.000013,-0.535105,0.25,0.002017
2,6,Paire B.,Bhambri Y.,1,1,1,False,True,0.500000,8.660254e-01,...,0.00320,-0.394797,0.008986,0.000019,0.186220,0.041436,0.000084,-0.535105,0.25,0.002017
3,7,Berrettini M.,Altmaier D.,1,1,0,False,False,-1.000000,-1.836970e-16,...,0.00220,-0.493123,0.003495,0.000008,0.525891,0.060407,0.000122,1.811315,1.00,0.002020
4,1,Molchanov D.,Mousley B.,1,1,1,False,False,0.500000,8.660254e-01,...,0.00280,3.755303,0.240773,0.000483,8.451724,0.503072,0.001009,-1.317245,0.00,0.002016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,9,Marcora R.,Duckworth J.,1,1,0,False,False,0.866025,5.000000e-01,...,0.00280,0.990705,0.086368,0.000174,0.186220,0.041436,0.000084,1.811315,1.00,0.002020
35830,5,Norrie C.,Thompson J.,2,1,1,False,False,-0.866025,-5.000000e-01,...,0.00246,-0.198145,0.019969,0.000041,-0.180268,0.020968,0.000043,0.247035,0.50,0.002018
35831,9,Rublev A.,Berrettini M.,1,1,0,False,False,-0.866025,5.000000e-01,...,0.00235,-0.493123,0.003495,0.000008,-0.493123,0.003495,0.000008,1.029175,0.75,0.002019
35832,10,Federer R.,Tsitsipas S.,1,1,1,False,False,-0.866025,5.000000e-01,...,0.00211,-0.555694,0.000000,0.000001,-0.519939,0.001997,0.000005,1.029175,0.75,0.002019


In [320]:
i = 0
for df in dataframes:
    print(" \n"+dataframe_names[i])
    lin_svc_pipeline(df)
    print(" \n"+dataframe_names[i])
    rfc_pipeline(df)
    print(" \n"+dataframe_names[i])
    logReg_pipeline(df)
    i+=1

 
df_hand_int_N_all
Linear Support Vector Classifier
              precision    recall  f1-score   support

           0       0.64      0.62      0.63      3622
           1       0.62      0.64      0.63      3545

    accuracy                           0.63      7167
   macro avg       0.63      0.63      0.63      7167
weighted avg       0.63      0.63      0.63      7167

Accuracy: 0.6295521138551695
Precision: 0.6213857064920895
Recall: 0.6425952045133991
F1 Score: 0.6318125086673139
 
df_hand_int_N_all
Random Forest Classifier
              precision    recall  f1-score   support

           0       0.70      0.71      0.71      3622
           1       0.70      0.69      0.70      3545

    accuracy                           0.70      7167
   macro avg       0.70      0.70      0.70      7167
weighted avg       0.70      0.70      0.70      7167

Accuracy: 0.7026649923259384
Precision: 0.7026949541284404
Recall: 0.6913963328631876
F1 Score: 0.6969998578131665
ROC-AUC Score: 0.7